In [21]:
import pandas as pd

In [22]:
data_path = './corona_scotland.xlsx'

In [23]:
df = pd.read_excel(data_path, 
                   dtype={'Total Tests': 'Int32',
                          'Total Positive Cases': 'Int32',
                          'Total Deaths': 'Int32'}
                   ).ffill()
df = df[df['Health Board'] != 'Golden Jubilee National Hospital']

In [24]:
df

,Date,Health Board,Number of Positive cases,Total Tests,Total Positive Cases,Total Deaths
0,2020-03-06,Tayside,1,1514,11,0
1,2020-03-06,Ayrshire & Arran,1,1514,11,0
2,2020-03-06,Forth Valley,2,1514,11,0
3,2020-03-06,Greater Glasgow and Clyde,1,1514,11,0
4,2020-03-06,Grampian,3,1514,11,0
...,...,...,...,...,...,...
451,2020-04-12,Lothian,915,31114,5912,566
452,2020-04-12,Orkney,5,31114,5912,566
453,2020-04-12,Shetland,43,31114,5912,566
454,2020-04-12,Tayside,803,31114,5912,566


## clean data

In [25]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        38
Lothian                          38
Forth Valley                     38
Grampian                         38
Fife                             38
Tayside                          38
Lanarkshire                      37
Shetland                         35
Borders                          33
Highland                         30
Dumfries and Galloway            28
Ayrshire and Arran               20
Ayrshire and Arran               15
Eileanan Siar (Western Isles)    12
Orkney                           12
Ayrshire & Arran                  3
Name: Health Board, dtype: int64

In [26]:
# clean up some names
df.loc[df['Health Board'] == 'Ayrshire\xa0and Arran', 'Health Board'] = 'Ayrshire and Arran' 
df.loc[df['Health Board'] == 'Ayrshire & Arran', 'Health Board'] = 'Ayrshire and Arran' 

In [27]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        38
Ayrshire and Arran               38
Lothian                          38
Forth Valley                     38
Grampian                         38
Fife                             38
Tayside                          38
Lanarkshire                      37
Shetland                         35
Borders                          33
Highland                         30
Dumfries and Galloway            28
Eileanan Siar (Western Isles)    12
Orkney                           12
Name: Health Board, dtype: int64

## table of number cases per region over time

In [28]:
df_cases = df[['Date', 'Health Board', 'Number of Positive cases']]

In [29]:
df_table = pd.pivot_table(df_cases, 
                          columns=['Health Board'], 
                          values=['Number of Positive cases'], 
                          index=['Date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)

In [30]:
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [31]:
df_table.to_pickle('corona_scotland_regions_ts.pk')

## table of national cases, tests, and deaths over time

In [32]:
df_national = (df.drop(columns=['Health Board', 'Number of Positive cases'])
                 .groupby('Date').first()
              )

In [33]:
df_national.index

DatetimeIndex(['2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
               '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
               '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
               '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
               '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
               '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
               '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06',
               '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [34]:
df_national.to_pickle('corona_scotland_total_stats.pk')

## Scotland Population Data

In [35]:
region_pops = {
    'Ayrshire and Arran': 369670,
    'Borders': 115270, 
    'Dumfries and Galloway': 148790,
    'Fife': 371910,
    'Forth Valley': 306070,
    'Grampian': 584550,
    'Greater Glasgow and Clyde': 1196335,
    'Highland': 321800,
    'Lanarkshire': 563185,
    'Lothian': 897770,
    'Orkney': 22190,
    'Shetland': 22990,
    'Tayside': 416080,
    'Eileanan Siar (Western Isles)': 26500,
    #'Scotland (Total)': 5438000,
}

In [36]:
df_pop = pd.DataFrame(region_pops.items(), columns=['Region', 'Pop']).set_index('Region').sort_index()

In [37]:
df_pop.loc['Scotland (Total)'] = 5438000

In [38]:
df_pop

,Pop
Region,
Ayrshire and Arran,369670
Borders,115270
Dumfries and Galloway,148790
Eileanan Siar (Western Isles),26500
Fife,371910
Forth Valley,306070
Grampian,584550
Greater Glasgow and Clyde,1196335
Highland,321800


In [39]:
df_pop[:14].sum()
## hmm missing few thousands??

Pop    5363110
dtype: int64

In [40]:
df_pop.to_pickle('scotland_population.pk')